In [ ]:
import torchvision  # for NMS
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

def setup_cfg():
    """
    Sets up and returns the Detectron2 configuration.
    """
    cfg = get_cfg()
    # Replace with the path to your YAML config file
    cfg.merge_from_file("config/pothole_detection_config.yaml")

    # Replace with the path to your trained weights
    cfg.MODEL.WEIGHTS = "./output/model_final.pth"

    # Confidence threshold for predictions
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.90

    # Your registered dataset name for validation
    cfg.DATASETS.TEST = ("my_val_dataset",)

    # Use GPU if available; otherwise use "cpu"
    cfg.MODEL.DEVICE = "cuda"

    return cfg


def perform_inference(image_path, cfg, iou_threshold=0.5):
    """
    Performs inference on a single image, applies Non-Maximum Suppression (NMS),
    and saves the annotated image with bounding boxes that do not overlap.
    
    Args:
        image_path (str): The path to the input image.
        cfg: The Detectron2 configuration object.
        iou_threshold (float): IoU threshold for NMS. 
                               Boxes with IoU >= this threshold are considered overlapping.
    """
    # Initialize the predictor
    predictor = DefaultPredictor(cfg)

    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Image {image_path} not found.")
        return

    # Perform prediction
    outputs = predictor(image)

    # Convert the detected instances to CPU for further processing
    instances = outputs["instances"].to("cpu")

    # If no instances detected, simply save the unmodified image
    if len(instances) == 0:
        output_image_path = os.path.splitext(image_path)[0] + "_output.jpg"
        cv2.imwrite(output_image_path, image)
        print(f"No instances detected. Output saved to: {output_image_path}")
        return

    # Extract boxes and scores
    boxes = instances.pred_boxes.tensor  # (N, 4)
    scores = instances.scores           # (N, )

    # Apply Non-Maximum Suppression
    keep_indices = torchvision.ops.nms(boxes, scores, iou_threshold)
    filtered_instances = instances[keep_indices]

    # Visualize results with the filtered instances
    v = Visualizer(
        image[:, :, ::-1],
        metadata=MetadataCatalog.get(cfg.DATASETS.TEST[0]),
        scale=0.8
    )
    out = v.draw_instance_predictions(filtered_instances)

    # Save or display the result
    output_image_path = os.path.splitext(image_path)[0] + "_output.jpg"
    cv2.imwrite(output_image_path, out.get_image()[:, :, ::-1])
    print(f"Output saved to: {output_image_path}")


if __name__ == "__main__":
    # Setup configuration
    cfg = setup_cfg()

    # Loop through some example images
    for i in range(5):
        # Construct the path to the test image
        image_path = f"dataset/test/images/test_image_{i}.jpg"
        
        # Perform inference with NMS
        perform_inference(image_path, cfg, iou_threshold=0.30)
